<a href="https://colab.research.google.com/github/TBab0/duth_cv4/blob/main/1sratch%20mod3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import datetime

In [2]:
drive.mount('/content/drive')
local_zip = '/content/drive/MyDrive/imagedb_btsd.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

train_dir = '/content/imagedb'
test_dir = '/content/imagedb_test'

Mounted at /content/drive


In [3]:
ts = 32
# train_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input, validation_split=0.2)
train_datagen  = ImageDataGenerator(rescale=1./255,
                                    validation_split=0.2)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    color_mode='grayscale',
                                                    target_size=(ts,ts),
                                                    subset='training', seed=1)      
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  train_datagen.flow_from_directory(train_dir,
                                                        batch_size=128,
                                                        class_mode='categorical',
                                                         color_mode='grayscale',
                                                         target_size=(ts,ts),
                                                        subset='validation', seed=1)

Found 2457 images belonging to 34 classes.
Found 599 images belonging to 34 classes.


In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(ts, ts, 1)),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(34, activation='softmax')
])
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 100)               313700    
                                                                 
 dense_1 (Dense)             (None, 34)                3434      
                                                                 
Total params: 317,550
Trainable params: 317,550
Non-trainable params: 0
__________________________________________________

In [5]:
callbacks = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)

history = model.fit(train_generator, validation_data=validation_generator, epochs=30, verbose=1, callbacks=callbacks)

Epoch 1/30
19/20 [===========================>..] - ETA: 0s - loss: 2.9878 - accuracy: 0.2598
Epoch 1: val_loss improved from inf to 2.37603, saving model to best_weights.hdf5
20/20 [==============================] - 9s 66ms/step - loss: 2.9617 - accuracy: 0.2658 - val_loss: 2.3760 - val_accuracy: 0.4357
Epoch 2/30
19/20 [===========================>..] - ETA: 0s - loss: 1.9272 - accuracy: 0.5608
Epoch 2: val_loss improved from 2.37603 to 1.53147, saving model to best_weights.hdf5
20/20 [==============================] - 1s 57ms/step - loss: 1.9137 - accuracy: 0.5629 - val_loss: 1.5315 - val_accuracy: 0.6711
Epoch 3/30
19/20 [===========================>..] - ETA: 0s - loss: 1.2098 - accuracy: 0.7359
Epoch 3: val_loss improved from 1.53147 to 1.00845, saving model to best_weights.hdf5
20/20 [==============================] - 1s 54ms/step - loss: 1.2005 - accuracy: 0.7387 - val_loss: 1.0084 - val_accuracy: 0.7846
Epoch 4/30
19/20 [===========================>..] - ETA: 0s - loss: 0.7744

In [6]:
test_datagen  = ImageDataGenerator(rescale=1./255)
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=32,
                                                         class_mode  = 'categorical',
                                                         color_mode='grayscale',
                                                         target_size=(ts,ts)) 
loss, acc = model.evaluate(test_generator)
print(acc)


Found 2149 images belonging to 34 classes.
68/68 [==============================] - 1s 13ms/step - loss: 0.2916 - accuracy: 0.9260
0.9260120987892151
